
# Tutorial: Promoting the Winner(s): Hyperparameter Tuning on Top Candidates

Selamat datang di subchapter 4.4! Sejauh ini, kita telah menyelenggarakan "turnamen" model dan, berdasarkan visualisasi *box plot*, kita telah mengidentifikasi juara kita: **Random Forest** dan **Gradient Boosting**.

Namun, model-model ini masih menggunakan pengaturan *default* mereka. Anggap saja mereka adalah mobil balap yang baru keluar dari pabrik. Untuk benar-benar memenangkan perlombaan, kita perlu membawanya ke pit stop dan menyetel mesinnya.

**Tujuan:** Di notebook ini, kita akan melakukan *hyperparameter tuning* pada model-model kandidat teratas kita untuk menemukan kombinasi pengaturan terbaik yang memberikan performa maksimal.




---
### 1. Tujuan Pembelajaran

Di akhir notebook ini, Anda akan dapat:

* Mendefinisikan ruang pencarian (grid) hyperparameter untuk model-model *ensemble*.
* Menggunakan `RandomizedSearchCV` untuk secara efisien menemukan kombinasi hyperparameter terbaik.
* Membandingkan performa model yang sudah di-*tuning* untuk memilih satu juara akhir.
* Memahami alur kerja untuk mempromosikan model dari kandidat menjadi solusi final.




---
### 2. Setup: Library dan Data

Kita akan menggunakan setup yang sama seperti sebelumnya, tetapi kali ini kita akan menambahkan `RandomizedSearchCV` ke dalam daftar impor kita.



In [2]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Model-model kandidat kita
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Muat data
housing = fetch_california_housing()
X, y = housing.data, housing.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



HTTPError: HTTP Error 403: Forbidden


---
### 3. Langkah 1: Siapkan Kandidat Final dan Pipeline-nya

Berdasarkan hasil dari subchapter 4.3, kita fokus pada dua model terbaik. Kita akan membuat pipeline untuk masing-masing.



In [ ]:
rf_pipeline = make_pipeline(
	StandardScaler(),
	RandomForestRegressor(random_state=42, n_jobs=-1)
)

gb_pipeline = make_pipeline(
	StandardScaler(),
	GradientBoostingRegressor(random_state=42)
)



---
### 4. Langkah 2: Definisikan Ruang Pencarian Hyperparameter

Ini adalah langkah kunci. Kita akan membuat *dictionary* yang berisi nama hyperparameter yang ingin kita *tuning* dan rentang nilai yang ingin kita coba. Kita akan menggunakan `RandomizedSearchCV` karena lebih cepat daripada `GridSearchCV` untuk ruang pencarian yang besar.

**Untuk Random Forest:**
* `n_estimators`: Jumlah pohon dalam hutan.
* `max_features`: Jumlah fitur yang dipertimbangkan di setiap *split*.
* `max_depth`: Kedalaman maksimum setiap pohon.
* `min_samples_split`: Jumlah minimum sampel yang dibutuhkan untuk memecah simpul.
* `min_samples_leaf`: Jumlah minimum sampel di setiap daun.



In [ ]:
# grid untuk random forest
rf_param_grid = {
	"randomforestregressor__n_estimators": [100,200,300,500],
	"randomforestregressor__max_features": ['sqrt','log2',1.0],
	"randomforestregressor__max_depth":[None,10,20,30],
	"randomforestregressor__min_samples_split":[2,5,10],
	"randomforestregressor__min_samples_leaf":[1,2,4]
}

# grid untuk gradient boosting
gb_param_grid = {
	"gradientboostingregressor__n_estimators": [100,200,300,500],
	"gradientboostingregressor__learning_rate": [0.01,0.05,0.1,0.2],
	"gradientboostingregressor__max_depth":[3,5,7],
	"gradientboostingregressor__subsample":[0.7,0.8,0.8,1.0]
}


---
### 5. Langkah 3: Jalankan Randomized Search CV

Kita akan menjalankan proses *tuning* untuk setiap model. `n_iter` mengontrol berapa banyak kombinasi acak yang akan dicoba.



In [ ]:
print("memulai tuning untuk random forest")
rs_rf = RandomizedSearchCV(
	estimator=rf_pipeline,
	param_distributions=rf_param_grid,
	n_iter=50,
	cv=5,
	scoring='r2',
	n_jobs=-1,
	random_state=42,
	verbose=1
)

rs_rf.fit(X_train,y_train)
print("Tuning untuk random forest selesai")

memulai tuning untuk random forest
Fitting 5 folds for each of 50 candidates, totalling 250 fits


/Users/faqihza/GitHub/FastCampus/Course 2/.env/lib/python3.13/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Tuning untuk random forest selesai


In [ ]:
print("memulai tuning untuk gradient boosting")
rs_gb = RandomizedSearchCV(
	estimator=gb_pipeline,
	param_distributions=gb_param_grid,
	n_iter=50,
	cv=5,
	scoring='r2',
	n_jobs=-1,
	random_state=42,
	verbose=1
)

rs_gb.fit(X_train,y_train)
print("Tuning untuk gradient boosting selesai")

memulai tuning untuk gradient boosting
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Tuning untuk gradient boosting selesai



---
### 6. Langkah 4: Bandingkan Juara yang Sudah Di-tuning

Sekarang kita bisa membandingkan skor terbaik dari kedua proses *tuning* untuk menentukan juara akhir.



In [ ]:
print("---- hasil hyperparameter tuning ----")
print("\nRandom forest")
print(f"skor r2 CV terbaik: {rs_rf.best_score_:.4f}")
print("Parameter terbaik:")

for param,value in rs_rf.best_params_.items():
	print(f"{param}: {value}")

print("---- hasil hyperparameter tuning ----")
print("\nGradient Boosting")
print(f"skor r2 CV terbaik: {rs_gb.best_score_:.4f}")
print("Parameter terbaik:")

for param,value in rs_gb.best_params_.items():
	print(f"{param}: {value}")

---- hasil hyperparameter tuning ----

Random forest
skor r2 CV terbaik: 0.8185
Parameter terbaik:
randomforestregressor__n_estimators: 500
randomforestregressor__min_samples_split: 2
randomforestregressor__min_samples_leaf: 1
randomforestregressor__max_features: log2
randomforestregressor__max_depth: None
---- hasil hyperparameter tuning ----

Gradient Boosting
skor r2 CV terbaik: 0.8397
Parameter terbaik:
gradientboostingregressor__subsample: 0.8
gradientboostingregressor__n_estimators: 500
gradientboostingregressor__max_depth: 7
gradientboostingregressor__learning_rate: 0.1



**Analisis Hasil:**

Setelah proses *tuning*, kita bisa melihat skor R² CV terbaik dari masing-masing model.

* **Peningkatan Performa:** Bandingkan skor ini dengan skor *default* yang kita dapatkan di subchapter sebelumnya. Kemungkinan besar, Anda akan melihat peningkatan, meskipun mungkin tidak drastis jika pengaturan *default*-nya sudah cukup baik.
* **Pemilihan Juara:** Berdasarkan skor CV terbaik, kita dapat memilih satu model sebagai juara akhir. Dalam banyak kasus, Gradient Boosting yang sudah di-*tuning* seringkali memiliki sedikit keunggulan. Namun, jika perbedaannya sangat kecil, Random Forest bisa menjadi pilihan yang lebih aman karena lebih robust.

Mari kita asumsikan, berdasarkan hasil di atas, **Gradient Boosting yang telah di-tuning** adalah pemenang kita.




---
### 7. Kesimpulan Tutorial

Kita telah berhasil mengambil model-model kandidat terbaik kita dan menyetelnya untuk mendapatkan performa maksimal. Proses ini adalah inti dari optimisasi model dalam *machine learning*.

Kita sekarang memiliki satu model juara (`rs_gb.best_estimator_`) yang siap untuk dievaluasi secara final.

Di subchapter selanjutnya dan terakhir, kita akan melakukan satu langkah terakhir yang krusial: melatih model juara ini pada **seluruh data pelatihan** dan mengujinya **satu kali** pada **data pengujian** yang selama ini kita simpan, untuk mendapatkan laporan performa akhir yang paling jujur.
